# API OpenWeatherMap



## Configuration de l'application:

- api_key


### Chargement de la configuration sinon création d'une configuration de base

In [7]:
# Chargement de la configuration ou création d"une configuration de base
from ConfigJSON import ConfigJSON

# Variable de configuration globale
config = ConfigJSON("config.json")

if config.exist_file():
    
    config.read_file()
    print( config )
    
else:
    print("Erreur pas de fichier de configuration config.json")
    
    api_key = input("Entrez votre clé d'API openWeather Map: ")
    config.set_config( "api_key", api_key )
    
    config.write_file()
    

ConfigJSON: [
	dossier: '.'
	fichier: 'config.json'
	config: {
		api_key: **secret**
	}
]



### Geocoding API

Documentation : https://openweathermap.org/api/geocoding-api

Faire une requete pour rechercher une ville:

In [12]:
from ConfigJSON import ConfigJSON

config = ConfigJSON("config.json")
config.read_file()

ville = input("Saisir une ville : ")
pays = input("Code pays (ISO 3166) : ")

url_api = "http://api.openweathermap.org/geo/1.0/direct?limit=20&appid=" + config.get_config("api_key")
url_api += "&q=" + ville
if len(pays) > 0:
    url_api += "," + pays

# Executer la requete
req = requests.get( url_api )
api_data = req.json()

#print(api_data)

geo_match = []

for index_geo, geo_loc in enumerate(api_data):
    
    local_name = ""
    if "local_names" in geo_loc:
            for key in geo_loc["local_names"]:
                if local_name == "" and key == "fr":
                    local_name = geo_loc["local_names"][key]
    geo_match.append( [geo_loc["name"], local_name, geo_loc["lat"], geo_loc["lon"], geo_loc["country"], geo_loc["state"] ] )

# Affichage des localisations trouvées
for index_geo, geo_loc in enumerate(geo_match):
    print(index_geo, geo_loc)
    

Saisir une ville : saint-étienne
Code pays (ISO 3166) : FR
0 ['Saint-Étienne', 'Saint-Étienne', 45.4401467, 4.3873058, 'FR', 'Auvergne-Rhône-Alpes']
1 ['Saint-Étienne-la-Cigogne', 'Saint-Étienne-la-Cigogne', 46.1148482, -0.5034125, 'FR', 'Nouvelle-Aquitaine']
2 ['Saint-Étienne', 'Saint-Étienne', 45.409604200000004, 4.484915566669622, 'FR', 'Auvergne-Rhône-Alpes']
3 ['Saint-Étienne', 'Saint-Étienne', 43.1587144, -0.8931555, 'FR', 'Nouvelle-Aquitaine']
4 ['Saint-Étienne', 'Saint-Étienne', 43.2419885, -1.1431591, 'FR', 'Nouvelle-Aquitaine']


#### enregistrement de la configuration d'une localisation trouvé

In [13]:
from ConfigJSON import ConfigJSON

config = ConfigJSON("config.json")
config.read_file()

index_geo = int( input("Numéro de localisation à paramétrer : ") )

config.set_config( "lat", geo_match[index_geo][2] )
config.set_config( "lon", geo_match[index_geo][3] )
    
config.write_file()

Numéro de localisation à paramétrer : 0


### Appel API météo actuelle

In [19]:
# Endpoint current weather data
# Documentation : https://openweathermap.org/current

from ConfigJSON import ConfigJSON

config = ConfigJSON("config.json")
config.read_file()

url_api = "http://api.openweathermap.org/data/2.5/weather?units=metric&lang=fr&appid=" + config.get_config("api_key")
url_api += "&lat=" + str( config.get_config("lat") )
url_api += "&lon=" + str( config.get_config("lon") )

# Executer la requete
req = requests.get( url_api )
api_data = req.json()

#print(api_data)

meteo = {}
meteo['timestamp'] = api_data['dt']
meteo['lieu'] = api_data["name"]
meteo['sunrise'] = api_data["sys"]["sunrise"]
meteo['sunset'] = api_data["sys"]["sunset"]
meteo['coord_lon'] = api_data["coord"]["lon"]
meteo['coord_lat'] = api_data["coord"]["lat"]
meteo['meteo_id'] = api_data["weather"][0]["id"]
meteo['meteo_group'] = api_data["weather"][0]["main"]
meteo['meteo_desc'] = api_data["weather"][0]["description"]
meteo['meteo_icon'] = api_data["weather"][0]["icon"]
meteo['vent_kph'] = api_data["wind"]["speed"] * 3.6
meteo['vent_dir'] = api_data["wind"]["deg"]
meteo['temperature'] = api_data["main"]["temp"]
meteo['temp_ressentie'] = api_data["main"]["feels_like"]
meteo['temp_min'] = api_data["main"]["temp_min"]
meteo['temp_max'] = api_data["main"]["temp_max"]
meteo['humidite'] = api_data["main"]["humidity"]
meteo['pression'] = api_data["main"]["pressure"]
meteo['visibilite'] = api_data["visibility"]
meteo['nuages'] = api_data["clouds"]["all"]

print(meteo)

{'timestamp': 1666542634, 'lieu': 'Saint-Étienne', 'sunrise': 1666505333, 'sunset': 1666543465, 'coord_lon': 4.3873, 'coord_lat': 45.4401, 'meteo_id': 800, 'meteo_group': 'Clear', 'meteo_desc': 'ciel dégagé', 'meteo_icon': '01d', 'vent_kph': 40.752, 'vent_dir': 190, 'temperature': 21.73, 'temp_ressentie': 21.16, 'temp_min': 21.02, 'temp_max': 22.17, 'humidite': 46, 'pression': 1011, 'visibilite': 10000, 'nuage': 0}


### Appel API prévisions météo

In [22]:
# Endpoint 5 day / 3 hour forecast data
# Documentation : https://openweathermap.org/forecast5
from ConfigJSON import ConfigJSON

config = ConfigJSON("config.json")
config.read_file()

url_api = "http://api.openweathermap.org/data/2.5/forecast?units=metric&lang=fr&appid=" + config.get_config("api_key")
url_api += "&lat=" + str( config.get_config("lat") )
url_api += "&lon=" + str( config.get_config("lon") )

# Executer la requete
req = requests.get( url_api )
api_data = req.json()

print(api_data)



{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1666548000, 'main': {'temp': 21.11, 'feels_like': 20.66, 'temp_min': 17.2, 'temp_max': 21.11, 'pressure': 1012, 'sea_level': 1012, 'grnd_level': 954, 'humidity': 53, 'temp_kf': 3.91}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'légère pluie', 'icon': '10n'}], 'clouds': {'all': 0}, 'wind': {'speed': 3.84, 'deg': 221, 'gust': 8.49}, 'visibility': 10000, 'pop': 0.62, 'rain': {'3h': 1.2}, 'sys': {'pod': 'n'}, 'dt_txt': '2022-10-23 18:00:00'}, {'dt': 1666558800, 'main': {'temp': 19.6, 'feels_like': 19.29, 'temp_min': 16.58, 'temp_max': 19.6, 'pressure': 1013, 'sea_level': 1013, 'grnd_level': 954, 'humidity': 64, 'temp_kf': 3.02}, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'pluie modérée', 'icon': '10n'}], 'clouds': {'all': 33}, 'wind': {'speed': 3.34, 'deg': 181, 'gust': 8.16}, 'visibility': 10000, 'pop': 0.62, 'rain': {'3h': 3.35}, 'sys': {'pod': 'n'}, 'dt_txt': '2022-10-23 21:00:00'}, {'dt': 1666569600, 'main'

In [26]:
#for key in api_data:
#    print(key, api_data[key])
#    print()

meteo_prev = []

lieu = api_data['city']['name']
sunrise = api_data["city"]["sunrise"]
sunset = api_data["city"]["sunset"]
coord_lon = api_data["city"]["coord"]["lon"]
coord_lat = api_data["city"]["coord"]["lat"]

for index, prev in enumerate( api_data['list'] ):
#    print(index, prev, type(prev), "\n")

    prevision = {}
    prevision['timestamp'] = prev['dt']
    prevision['date'] = prev['dt_txt']
    if prev['sys']['pod'] == "n":
        prevision['jour_nuit'] = "nuit"
    elif prev['sys']['pod'] == "d":
        prevision['jour_nuit'] = "jour"
    else:
        prevision['jour_nuit'] = ""

    prevision['lieu'] = lieu
    prevision['sunrise'] = sunrise
    prevision['sunset'] = sunset
    prevision['coord_lon'] = coord_lon
    prevision['coord_lat'] = coord_lat
        
    prevision['meteo_id'] = prev['weather'][0]['id']
    prevision['meteo_desc'] = prev['weather'][0]['description']
    prevision['meteo_group'] = prev['weather'][0]['main']
    prevision['meteo_icon'] = prev['weather'][0]['icon']
    
    prevision['temperature'] = round( prev['main']['temp'], 1 )
    prevision['temp_ressentie'] = round( prev['main']['feels_like'], 1 )
    prevision['temp_max'] = round( prev['main']['temp_max'], 1 )
    prevision['temp_min'] = round( prev['main']['temp_min'], 1 )
    
    prevision['pression'] = prev['main']['grnd_level']
    prevision['pression_mer'] = prev['main']['sea_level']
    prevision['humidite'] = prev['main']['humidity']
    prevision['nuages'] = prev['clouds']['all']
    prevision['vent_kph'] = round( prev["wind"]["speed"] * 3.6, 1 )
    prevision['vent_dir'] = prev["wind"]["deg"]
    prevision['vent_rafale'] = round( prev["wind"]["gust"] * 3.6, 1 )
    prevision['visibilite'] = prev["visibility"]
    prevision['pluie_probalite'] = int( prev["pop"] * 100 )
    if "rain" in prev:
        prevision['pluie_volume'] = round( prev["rain"]["3h"] )
    else:
        prevision['pluie_volume'] = 0
    if "snow" in prev:
        prevision['neige_volume'] = prev["snow"]["3h"]
    else:
        prevision['neige_volume'] = 0
    
    meteo_prev.append(prevision)
    print(prevision, "\n")
    

{'timestamp': 1666548000, 'date': '2022-10-23 18:00:00', 'jour_nuit': 'nuit', 'lieu': 'Saint-Étienne', 'sunrise': 1666505333, 'sunset': 1666543465, 'coord_lon': 4.3873, 'coord_lat': 45.4401, 'meteo_id': 500, 'meteo_desc': 'légère pluie', 'meteo_group': 'Rain', 'meteo_icon': '10n', 'temperature': 21.1, 'temp_ressentie': 20.7, 'temp_max': 21.1, 'temp_min': 17.2, 'pression': 954, 'pression_mer': 1012, 'humidite': 53, 'nuages': 0, 'vent_kph': 13.8, 'vent_dir': 221, 'vent_rafale': 30.6, 'visibilite': 10000, 'pluie_probalite': 62, 'pluie_volume': 1, 'neige_volume': 0} 

{'timestamp': 1666558800, 'date': '2022-10-23 21:00:00', 'jour_nuit': 'nuit', 'lieu': 'Saint-Étienne', 'sunrise': 1666505333, 'sunset': 1666543465, 'coord_lon': 4.3873, 'coord_lat': 45.4401, 'meteo_id': 501, 'meteo_desc': 'pluie modérée', 'meteo_group': 'Rain', 'meteo_icon': '10n', 'temperature': 19.6, 'temp_ressentie': 19.3, 'temp_max': 19.6, 'temp_min': 16.6, 'pression': 954, 'pression_mer': 1013, 'humidite': 64, 'nuages': 